Задача

a) Узнать курсы доллара, евро и юаня за 100 последних календарных дней
R01335 - доллар, R01239 - евро, R01375 - юань

Даты должны отсчитываться от даты запуска програмы (формат даты %d/%m/%Y)
date_req1 - дата начала, date_req2 - дата конца

б) Выгрузить курсы валют в виде таблицы с 4 колонками: дата, доллар, евро, юань (В любом формате)

в) (сделаем вместе) Изменить в таблице формат дат на дату вида "08 агуста 2024"

In [2]:
'https://cbr.ru/scripts/XML_dynamic.asp?date_req1=02/03/2001&date_req2=14/03/2001&VAL_NM_RQ=R01235'

'https://cbr.ru/scripts/XML_dynamic.asp?date_req1=02/03/2001&date_req2=14/03/2001&VAL_NM_RQ=R01235'

In [3]:
from xml.etree import ElementTree
import datetime
import requests

In [4]:
d2 = datetime.date.today()
d2_ddmmyyyy = d2.strftime('%d/%m/%Y')

d1 = d2 - datetime.timedelta(days=100)
d1_ddmmyyyy = d1.strftime('%d/%m/%Y')

print(d1_ddmmyyyy)

26/05/2024


In [5]:
currencies = {
    "доллар": "R01335",
    "евро": "R01239",
    "юань": "R01375"
}

Запрос - 1-й способ: вписываем переменные в строку

In [6]:
url = f'https://cbr.ru/scripts/XML_dynamic.asp?date_req1={d1_ddmmyyyy}&date_req2={d2_ddmmyyyy}&VAL_NM_RQ={currencies["юань"]}'
resp = requests.get(url)
resp.text

'<?xml version="1.0" encoding="windows-1251"?><ValCurs ID="R01375" DateRange1="26.05.2024" DateRange2="03.09.2024" name="Foreign Currency Market Dynamic"><Record Date="28.05.2024" Id="R01375"><Nominal>1</Nominal><Value>12,1903</Value><VunitRate>12,1903</VunitRate></Record><Record Date="29.05.2024" Id="R01375"><Nominal>1</Nominal><Value>12,1628</Value><VunitRate>12,1628</VunitRate></Record><Record Date="30.05.2024" Id="R01375"><Nominal>1</Nominal><Value>12,2752</Value><VunitRate>12,2752</VunitRate></Record><Record Date="31.05.2024" Id="R01375"><Nominal>1</Nominal><Value>12,3650</Value><VunitRate>12,365</VunitRate></Record><Record Date="01.06.2024" Id="R01375"><Nominal>1</Nominal><Value>12,4028</Value><VunitRate>12,4028</VunitRate></Record><Record Date="04.06.2024" Id="R01375"><Nominal>1</Nominal><Value>12,3011</Value><VunitRate>12,3011</VunitRate></Record><Record Date="05.06.2024" Id="R01375"><Nominal>1</Nominal><Value>12,2248</Value><VunitRate>12,2248</VunitRate></Record><Record Date="

Запрос - 2-й способ - через словарь с параметрами

In [7]:
p = {
    'date_req1': d1_ddmmyyyy,
    'date_req2': d2_ddmmyyyy,
    'VAL_NM_RQ': currencies["евро"]
}

endpoint = 'https://www.cbr.ru/scripts/XML_dynamic.asp'

resp = requests.get(endpoint, params=p)
resp.text

'<?xml version="1.0" encoding="windows-1251"?><ValCurs ID="R01239" DateRange1="26.05.2024" DateRange2="03.09.2024" name="Foreign Currency Market Dynamic"><Record Date="28.05.2024" Id="R01239"><Nominal>1</Nominal><Value>96,3034</Value><VunitRate>96,3034</VunitRate></Record><Record Date="29.05.2024" Id="R01239"><Nominal>1</Nominal><Value>96,2383</Value><VunitRate>96,2383</VunitRate></Record><Record Date="30.05.2024" Id="R01239"><Nominal>1</Nominal><Value>96,8910</Value><VunitRate>96,891</VunitRate></Record><Record Date="31.05.2024" Id="R01239"><Nominal>1</Nominal><Value>97,1347</Value><VunitRate>97,1347</VunitRate></Record><Record Date="01.06.2024" Id="R01239"><Nominal>1</Nominal><Value>97,7908</Value><VunitRate>97,7908</VunitRate></Record><Record Date="04.06.2024" Id="R01239"><Nominal>1</Nominal><Value>96,8721</Value><VunitRate>96,8721</VunitRate></Record><Record Date="05.06.2024" Id="R01239"><Nominal>1</Nominal><Value>96,5630</Value><VunitRate>96,563</VunitRate></Record><Record Date="0

Про аутентификацию
1. Анонимные запросы (редко бывает)
2. Нужно передавать вместе с запросами ваш ключ (API key)
   * либо как еще параметр в запросе, например: https://cbr.ru/scripts/XML_dynamic.asp?date_req1=02/03/2001&date_req2=14/03/2001&VAL_NM_RQ=R01235&api_key=12432525thjuhfg4597234u312u34234r2
   * либо в заголовке (параметр headers в библиотеке requests), например:
   
   requests.get('https://meteostat.p.rapidapi.com/stations/meta?id=10637', headers={'x-rapidapi-key': '12432525thjuhfg4597234u312u34234r2'

})
3. Нужно передать токен, который сначала надо получить на сервере аутентификации (например, если используете OAuth2)

In [8]:
currencies.items()

dict_items([('доллар', 'R01335'), ('евро', 'R01239'), ('юань', 'R01375')])

In [15]:
endpoint = 'https://www.cbr.ru/scripts/XML_dynamic.asp'

data = {}

for currency_name, currency_code in currencies.items():
    p = {
        'date_req1': d1_ddmmyyyy,
        'date_req2': d2_ddmmyyyy,
        'VAL_NM_RQ': currency_code
    }
    resp = requests.get(endpoint, params=p)
    #resp.text
    print(currency_name)

    # В этот раз передаем не файл, а переменную с текстом
    valcurs = ElementTree.fromstring(resp.text)
    #print(type(valcurs))

    tmp = {}
    for record in valcurs:
        date = record.attrib['Date']
        price = float(record.find['Value'].text.replace(',', '.'))
        #print(date, price)
        tmp[date] = price
    data[currency_name] = tmp
print(data)

доллар


TypeError: 'builtin_function_or_method' object is not subscriptable

Теперь пункт б) задачи

Составляем таблицу

In [16]:
data['доллар'].keys()

KeyError: 'доллар'